# Train a binary classification model

## Install required packages

In [ ]:
!pip install --upgrade pycaret scikit-plot

## Get workspace

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

## Setup mlflow tracking

In [ ]:
import mlflow

mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

# azure ml settings
experiment_name = "automl-with-pycaret-tutorial"

## Load a pandas.DataFrame

In [ ]:
from pycaret.datasets import get_data

df = get_data("credit")
df

In [ ]:
df.shape

## Split data

In [ ]:
data = df.sample(frac=0.95, random_state=42)
data_unseen = df.drop(data.index)
data.reset_index(inplace=True, drop=True)
data_unseen.reset_index(inplace=True, drop=True)
print("Data for modeling: " + str(data.shape))
print("Unseen data for predictions: " + str(data_unseen.shape))

## Import PyCaret

The `setup()` function initializes the environment in pycaret and creates the transformation pipeline to prepare the data for modeling and deployment. 

`setup()` must be called before executing any other function in pycaret. 

It takes two mandatory parameters: a `pandas.DataFrame` and the name of the target column. All other parameters are optional.

In [ ]:
from pycaret.classification import *

## Check supported algorithms

## Setup PyCaret

Refer to the [PyCaret documentation](https://pycaret.readthedocs.io/en/stable/) for details.

In [ ]:
exp = setup(
    data=data,
    target="default",
    log_experiment=True,
    experiment_name=experiment_name,
    log_plots=True,
    log_profile=True,
    silent=True,  # set to False for interactively setting data types
)

In [ ]:
models()

## Run AutoML

In [ ]:
%%time
best_model = compare_models()

In [ ]:
print(best_model)

## Evaluate model

In [ ]:
evaluate_model(best_model)

## Test model

In [ ]:
unseen_predictions = predict_model(best_model, data=data_unseen)
unseen_predictions.head()

In [ ]:
from pycaret.utils import check_metric

check_metric(
    unseen_predictions.default,
    unseen_predictions.Label.astype(int),
    "Accuracy",
)